In [1]:
!pip install lxml bs4 html5lib

     |████████████████████████████████| 6.3 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 90.4 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=136c5e01ae9d7848ffc7d71b1d7172dcf022a8c8379a0d1c272aae66c643d28d
  Stored in directory: /tmp/pip-ephem-wheel-cache-0gay8b60/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import urllib
import re
from bs4 import BeautifulSoup
import time

In [3]:
file_path = "art/wikiart"
base_url = "https://www.wikiart.org"

In [4]:
# iterate through all artists by last name alphabetically
for c in range(ord('l'), ord('z')+1):
    char = chr(c)
    artist_list_url = base_url + '/en/Alphabet/' + char + '/text-list'

    genre_soup = BeautifulSoup(urllib.request.urlopen(artist_list_url), "lxml")
    artist_list_main = genre_soup.find("main")
    lis = artist_list_main.find_all("li")

    # for each list element
    for li in lis: 
        born = 0
        died = 0

        # get the date range
        for line in li.text.splitlines():
            if line.startswith(",") and "-" in line:
                parts = line.split('-')
                if len(parts) == 2:
                    born = int(re.sub("[^0-9]", "",parts[0]))
                    died = int(re.sub("[^0-9]", "",parts[1]))

        # look for artists who may have created work that could in public domain
        if born>1850 and died>0 and (born<1900 or died<1950):
            link = li.find("a")
            artist = link.attrs["href"]

            if artist == "/en/salvador-dali": # skip Dali
                continue

            # get the artist's main page
            artist_url = base_url + artist
            artist_soup = BeautifulSoup(urllib.request.urlopen(artist_url), "lxml")

            # only look for artists with the word abstract on their main page
            if "Abstract" in artist_soup.text or "abstract" in artist_soup.text or "Avant-garde" \
                in artist_soup.text or "avant-garde" in artist_soup.text:
                print('\n\n' + artist + " " + str(born) + " - " + str(died))

                # get the artist's web page for the artwork
                url = base_url + artist + '/all-works/text-list'
                artist_work_soup = BeautifulSoup(urllib.request.urlopen(url), "lxml")

                # get the main section
                artist_main = artist_work_soup.find("main")
                image_count = 0
                artist_name = artist.split("/")[2]

                # get the list of artwork
                lis = artist_main.find_all("li")

                # for each list element
                for li in lis:
                    link = li.find("a")

                    if link != None:
                        painting = link.attrs["href"]

                        # get the painting
                        url = base_url + painting
                        print('paiting ', url)

                        try:
                            painting_soup = BeautifulSoup(urllib.request.urlopen(url), "lxml")

                        except:
                            print("error retreiving page")
                            continue

                        # check the copyright
                        if "Public domain" in painting_soup.text:

                            #check the genre
                            genre = painting_soup.find("span", {"itemprop":"genre"})
                            if genre != None and genre.text == "abstract":

                                # get the url
                                og_image = painting_soup.find("meta", {"property":"og:image"})
                                image_url = og_image["content"].split("!")[0] # ignore the !Large.jpg at the end
                                print('public domain paiting ', image_url)

                                save_path = file_path + "/" + artist_name + "_" + str(image_count) + ".jpg"

                                #download the file
                                try:
                                    print("downloading to " + save_path)
                                    time.sleep(0.2)  # try not to get a 403                    
                                    urllib.request.urlretrieve(image_url, save_path)
                                    image_count = image_count + 1
                                except Exception as e:
                                    print("failed downloading " + image_url, e) 



/en/vajda-lajos 1908 - 1941
paiting  https://www.wikiart.org/en/vajda-lajos/lel-t-s-a-templomdombr-l-1929
paiting  https://www.wikiart.org/en/vajda-lajos/montage-with-the-portrait-of-lajos-szab-1930
paiting  https://www.wikiart.org/en/vajda-lajos/tiger-and-lillies-1933
paiting  https://www.wikiart.org/en/vajda-lajos/chinese-execution-1933
paiting  https://www.wikiart.org/en/vajda-lajos/gustave-fleury-1933
paiting  https://www.wikiart.org/en/vajda-lajos/tolstoj-and-gandhi-1933
paiting  https://www.wikiart.org/en/vajda-lajos/still-life-on-a-table-of-horseshoe-form-1934
paiting  https://www.wikiart.org/en/vajda-lajos/table-still-life-with-armchair-1934
paiting  https://www.wikiart.org/en/vajda-lajos/black-self-portrait-1935
paiting  https://www.wikiart.org/en/vajda-lajos/dark-self-portrait-1935
paiting  https://www.wikiart.org/en/vajda-lajos/self-portrait-in-a-mask-1935
paiting  https://www.wikiart.org/en/vajda-lajos/houses-in-szentendre-with-blue-sky-1935
paiting  https://www.wikiart.o